In [1]:
import os
import sys
import numpy as np
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
import pixel as pix
import utilities as util
import tile
import importlib
import resnet as rs
import keras
from keras.optimizers import Adam

Using TensorFlow backend.


In [2]:
landsat, sentinel, dem, label = util.load_data()
tile_size = 64
num_classes = util.get_class_count()
pixel_gen = pix.pixel_gen(landsat, sentinel, dem, label, tile_size, num_classes)
tile_gen = tile.tile_gen(landsat, sentinel, dem, label, tile_size, num_classes)

In [3]:
os.environ["CUDA_VISIBLE_DEVICES"]="3";

In [4]:
pixels = pixel_gen.gen_pixels(1000000, balanced=True)
train_px, test_px, val_px = pixel_gen.train_val_test_split(pixels, 0.7, 0.7)
pixel_gen.print_balance()

pixels generated 742767
train:519936 val:155981 test:113645
Water:57407.0
Snow/Ice:0.0
Open Space Developed:62500.0
Low Intensity Developed:0.0
Medium Intensity Developed:62500.0
High Intensity Developed:0.0
Barren Land:53980.0
Deciduous Forest:50144.0
Evergreen Forest:62500.0
Mixed Forest:50420.0
Scrub/Shrub:53823.0
Grassland / Herbaceous:51993.0
Pasture/Hay:50000.0
Cultivated Land:62500.0
Woody Wetland:62500.0
Emergent Herbaceous Wetlands:62500.0


In [ ]:
epochs = 3
batch_size = 25
input_shape = tile_gen.get_tile_shape(reshape=True)
print(input_shape)

(10, 64, 64)


In [ ]:
model = rs.ResnetBuilder.build_resnet_101(input_shape,num_classes)
opt = Adam()
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit_generator(generator=tile_gen.tile_generator(train_px, batch_size), 
                    steps_per_epoch=len(train_px) // batch_size, epochs=epochs, verbose=1,
                    validation_data=tile_gen.tile_generator(val_px, batch_size),
                    validation_steps=len(val_px) // batch_size)

Epoch 1/3
 1105/20797 [>.............................] - ETA: 4:01:47 - loss: 5.3780 - acc: 0.3393

# Test Pixels

In [ ]:
tile_gen.evaluate_cnn(model, test_px)

# Test Pixels on specific tiles

In [ ]:
pixel_buckets = { 0: list(), 1: list(), 2: list(), 3: list(), 4: list()}
for pix in test_px:
    pixel_buckets[pix[1]].append(pix)

# Tile 0

In [ ]:
tile_gen.evaluate_cnn(model, pixel_buckets[0][:5000])

# Tile 1

In [ ]:
tile_gen.evaluate_cnn(model, pixel_buckets[1])

# Tile 2

In [ ]:
tile_gen.evaluate_cnn(model, pixel_buckets[2])

# Tile 3

In [ ]:
tile_gen.evaluate_cnn(model, pixel_buckets[3])

# Tile 4

In [ ]:
tile_gen.evaluate_cnn(model, pixel_buckets[4])